Welcome to the `pymacnet` demo! Here we will walk through the basic functionality of the module. 

----

 Before we can talk to a Maccor cycler we must have a Maccor cycler to talk to. Testing software functionality on a real cycler is dangerous so we've created a submodule `maccorspoofer` to emulate some of the behavior of the Maccor software.  

In [ ]:
import pymacnet
import pymacnet.maccorspoofer

A `MaccorSpoofer` class instance requires a configuration dictionary to create:

In [ ]:
maccorspoofer_config = {
    "server_ip": "127.0.0.1", # The IP address to host on. 
    "json_port": 1234, # The JSON communication port.
    "tcp_port": 5678,  # The TCP communication port.
    "num_channels": 128 # The number of channels on our fake Maccor cycler.
}
maccor_spoofer = pymacnet.maccorspoofer.MaccorSpoofer(maccorspoofer_config)
maccor_spoofer.start()

Wunderbar! Now let's start with creating a our `CyclerInterface` class instance. Like our `MaccorSpoofer`, a class instance of `CyclerInterface` requires a configuration dictionary on instantiation. The config values are as follows:

- `server_ip` -> The IP address of the Maccor server. Use 127.0.0.1 if running on the same machine as the server.
- `json_msg_port` -> The port to communicate through with JSON commands. Default set to 57570.
- `bin_msg_port` -> The port to communicate through with binary commands. Default set to 57560.

Note need to make sure the we have the same `server_ip`, `json_server_port`, and `tcp_server_port` specified for our `MaccorSpoofer` instance.

In [ ]:
cycler_interface_config = {
    'server_ip': maccorspoofer_config['server_ip'],
    'json_msg_port': maccorspoofer_config['json_port'],
    'bin_msg_port': maccorspoofer_config['tcp_port'],
    'msg_buffer_size_bytes': 4096
}
cycler_interface = pymacnet.CyclerInterface(cycler_interface_config)

The `CyclerInterface` allows us to get system wide information, like software version info:

In [ ]:
cycler_interface.read_system_info()

Or general system infromation:

In [ ]:
cycler_interface.read_general_info()

We can also use the `CyclerInterface` to read the status of all channels at once:

In [ ]:
cycler_interface.read_all_channel_statuses()

Now that we've demonstrated how a the `CyclerInterface` works, we'll look at a `ChannelInterface` class instance. Like our `CyclerInterface`, a class instance of `ChannelInterface` requires a configuration dictionary on instantiation. Note that individual `ChannelInterface` instances work with a single channel. The config values are as follows:

- `channel` -> The cycler channel to be targeted for all operations.
- `test_name` -> The test name to be used if a test is started. If left blank, Maccor will generate a unique random name for any started tests. Note that Maccor requires unique test names for each test.
- `test_procedure` -> The test procedure to be used, if starting a test with a procedure. Not needed with direct control.
- `c_rate_ah` -> The capacity value to be referenced when setting "C" values within a Maccor schedule. Units of amp-hours. Ignored if not used anywhere in the test.
- `v_max_safety_limit_v` -> Upper voltage safety limit for the channel. Units of volts.
- `v_min_safety_limit_v` -> Lower voltage safety limit for the channel. Units of volts.
- `i_max_safety_limit_a` -> Upper current safety limit for the channel. Units of amps.
- `i_min_safety_limit_a` -> Lower current safety limit for the channel. Units of amps.
- `v_max_v` -> Upper voltage limit used for charge/CV limits. Units of volts. Only used with direct control.
- `v_min_v` -> Lower voltage limit used for discharge limit. Units of volts. Only used with direct control.
- `data_record_time_s` -> How often data points are taken during direct control tests. Zero turns off. Used only for direct control.
- `data_record_voltage_delta_vbys` -> The dV/dt at which data points are taken during direct control tests. Zero disables. Used only for direct control.
- `data_record_current_delta_abys` -> The dI/dt at which data points are taken during direct control tests. Zero disables. Used only for direct control.
- `server_ip` -> The IP address of the Maccor server. Use 127.0.0.1 if running on the same machine as the server.
- `json_server_port` -> The port to communicate through with JSON commands. Default set to 57570.
- `tcp_server_port` -> The port to communicate through with TCP commands. Default set to 57560.

In [ ]:
if 'cycler_interface' in globals():
    del cycler_interface

channel_interface_config_channel_1 = {
    'server_ip': maccorspoofer_config['server_ip'],
    'json_msg_port': maccorspoofer_config['json_port'],
    'bin_msg_port': maccorspoofer_config['tcp_port'],
    'msg_buffer_size_bytes': 4096,
    'channel': 1,                               
    'test_name': 'pymacnet_procedure_control', 
    'test_procedure': 'test_procedure_1',
    'c_rate_ah': 1,
    'v_max_safety_limit_v': 4.2,
    'v_min_safety_limit_v': 2.9,
    'i_max_safety_limit_a': 2.0,
    'i_min_safety_limit_a': -2.0,
    'v_max_v': 4.2,
    'v_min_v': 3.0,
    'data_record_time_s': 1,
    'data_record_voltage_delta_vbys': 1,
    'data_record_current_delta_abys': 1,
}
channel_1_interface = pymacnet.ChannelInterface(channel_interface_config_channel_1)

`ChannelInterface` is actually a child class of `CyclerInterface`, so we have all the same methods available to us:

In [ ]:
channel_1_interface.read_general_info()

Great. Now that we have an interface for the channel we can start reading from and controlling the specific cycler channel. We'll start by reading the status, which gives relevant channel specific cycler readings:

In [ ]:
channel_1_status = channel_1_interface.read_status()
print(channel_1_status)

The `MaccorSpoofer` allows us to update readings on mocked channel as we see fit. This can be helpful for testing:

In [ ]:
channel_1_updated_status = {'Voltage':3.72,'Current':0.00,'Cycle':5}
result = maccor_spoofer.update_channel_status( 
    channel=(channel_1_interface.get_channel_number()-1), # On the server side channels are zero index.
    updated_status=channel_1_updated_status
)
print("Success = " + str(result))
channel_1_status = channel_1_interface.read_status()
print(channel_1_status)


We can also get auxiliary values in a similar fashion. Here we just get a list of the auxiliary readings:

In [ ]:
channel_1_aux = channel_1_interface.read_aux()
print(channel_1_aux)

Now if we wanted to start a test with the test procedure defined in the config we can do the following:

In [ ]:
result = channel_1_interface.start_test_with_procedure()
print("Success = " + str(result))

Note there are many errors that can come up when trying to start a test and `MaccorSpoofer` does not cover all of the potential conflicts at this time. Be careful!

Once a test is running we can continue to `read_status()` and do things like set channel variables. This can be used for cool things like setting variable charge and discharge current:

In [ ]:
result = channel_1_interface.set_channel_variable(var_num = 1, var_value = 25)
print("Success = " + str(result))

The `MaccorInterface` also has methods to enable direct control which allows for controlling the cycler channel directly without a test procedure. **This is potentially very dangerous and should only be attempted if you know what you are doing and have thoroughly tested your code**. Details on the direct control methods can be found in the `docs/` directory